In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x : '%.4f' % x)

In [2]:
df = pd.read_excel('../data/datos_finales.xlsx')

1️⃣ KPIs para columnas categóricas

In [3]:
categorical_cols = ['invoiceno', 'region', 'country', 'item_type', 'sales_channel', 'order_priority', 'ship_day']

cat_summary = {}
for col in categorical_cols:
    cat_summary[col] = {
        'Unique_values': df[col].nunique(),
        'Top_value': df[col].mode()[0],
        'Top_count': df[col].value_counts().iloc[0],
        'Percentage_top': df[col].value_counts(normalize=True).iloc[0]*100
    }

cat_summary_df = pd.DataFrame(cat_summary).T
print("===== KPIs CATEGÓRICOS =====")
print(cat_summary_df)


===== KPIs CATEGÓRICOS =====
               Unique_values           Top_value Top_count Percentage_top
invoiceno               3335              537434       675         0.9001
region                     7  sub-saharan africa     19547        26.0661
country                  185               sudan       465         0.6201
item_type                 12           baby food      6311         8.4158
sales_channel              2             offline     37560        50.0867
order_priority             4                   l     18794        25.0620
ship_day                   7              Monday     10848        14.4659


2️⃣ KPIs para columnas numéricas

In [4]:
numeric_cols = ['units_sold', 'unit_price', 'unit_cost', 'total_revenue', 'total_cost', 'total_profit']

num_summary = df[numeric_cols].agg(['sum', 'mean', 'median', 'std', 'min', 'max'])
print("\n===== KPIs NUMÉRICOS =====")
print(num_summary)

# Margen promedio por unidad
df['unit_margin'] = df['unit_price'] - df['unit_cost']
print("\nMargen promedio por unidad:", df['unit_margin'].mean())

# Margen promedio por pedido
df['order_margin'] = df['total_profit'] / df['total_revenue']
print("Margen promedio por pedido:", df['order_margin'].mean())


===== KPIs NUMÉRICOS =====
           units_sold      unit_price       unit_cost      total_revenue  \
sum    374846570.0000 1997094802.0000 1406655412.0000 9992556444320.0000   
mean        4998.6207      26631.4816      18757.9065     133251852.8380   
median      5006.0000      20570.0000      11711.0000      78704407.0000   
std         2882.1819      21683.9361      17556.5770     146885444.2768   
min            1.0000        933.0000        692.0000          1866.0000   
max        10000.0000      66827.0000      52496.0000     668069519.0000   

               total_cost       total_profit  
sum    7040268722951.0000 2952287721369.0000  
mean        93882767.3417      39369085.4963  
median      46424070.5000      28306036.0000  
std        114965395.7814      37855879.4327  
min             1384.0000           482.0000  
max        524907504.0000     173870000.0000  

Margen promedio por unidad: 7873.575010001334
Margen promedio por pedido: 0.3439542914968468


3️⃣ KPIs de clientes y pedidos

In [5]:
total_customers = df['customerid'].nunique()
total_orders = df['order_id'].nunique()
avg_orders_per_customer = total_orders / total_customers

print("\n===== KPIs DE CLIENTES Y PEDIDOS =====")
print("Clientes únicos:", total_customers)
print("Pedidos únicos:", total_orders)
print("Promedio de pedidos por cliente:", avg_orders_per_customer)



===== KPIs DE CLIENTES Y PEDIDOS =====
Clientes únicos: 1341
Pedidos únicos: 74990
Promedio de pedidos por cliente: 55.92095451155854


4️⃣ KPIs de fechas

In [6]:
# Tiempo promedio de envío
df['order_date'] = pd.to_datetime(df['order_date'])
df['ship_date'] = pd.to_datetime(df['ship_date'])
df['shipping_time_days'] = (df['ship_date'] - df['order_date']).dt.days

print("\n===== KPIs DE TIEMPO DE ENVÍO =====")
print("Tiempo promedio de envío (días):", df['shipping_time_days'].mean())
print("Tiempo máximo de envío (días):", df['shipping_time_days'].max())
print("Tiempo mínimo de envío (días):", df['shipping_time_days'].min())


===== KPIs DE TIEMPO DE ENVÍO =====
Tiempo promedio de envío (días): 25.043792505667422
Tiempo máximo de envío (días): 50
Tiempo mínimo de envío (días): 0


5️⃣ KPIs combinados

In [7]:
print("\n===== KPIs COMBINADOS =====")
# Ingresos totales por región
revenue_by_region = df.groupby('region')['total_revenue'].sum()
print("\nIngresos por región:\n", revenue_by_region)

# Beneficio promedio por tipo de producto
profit_by_item = df.groupby('item_type')['total_profit'].mean()
print("\nBeneficio promedio por tipo de producto:\n", profit_by_item)

# Unidades vendidas por mes
units_by_month = df.groupby('ship_month')['units_sold'].sum()
print("\nUnidades vendidas por mes:\n", units_by_month)

# Margen total por país
profit_by_country = df.groupby('country')['total_profit'].sum()
print("\nMargen total por país:\n", profit_by_country)


===== KPIs COMBINADOS =====

Ingresos por región:
 region
asia                                 1436182915934
australia and oceania                 810017642474
central america and the caribbean    1107175687336
europe                               2554909696330
middle east and north africa         1257845567083
north america                         223304217481
sub-saharan africa                   2603120717682
Name: total_revenue, dtype: int64

Beneficio promedio por tipo de producto:
 item_type
baby food         47861401.8488
beverages          7857505.7021
cereal            44412355.7602
clothes           36950156.7568
cosmetics         86462509.3810
fruits             1191370.2022
household         82887450.8196
meat              28753451.9915
office supplies   63336550.4766
personal care     12503661.7395
snacks            27722560.1238
vegetables        31245716.0718
Name: total_profit, dtype: float64

Unidades vendidas por mes:
 ship_month
1     30875708
2     29583718
3     33